In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install PyPDF2

In [ ]:
!pip install bs4

In [ ]:
!pip install fake_useragent

In [ ]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

In [ ]:
# Basic libraries
import pandas as pd
import numpy as np
import nltk

# Importing for scraping
from bs4 import BeautifulSoup 
import fake_useragent

import random
import time         # Tracking time
import requests     # HTTP requests
import re           # String manipulation

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
import requests
from lxml.html import fromstring
def get_proxies():
    # Function that return a list of proxies
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = []
    for i in parser.xpath('//tbody/tr'):
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            #Grabbing IP and corresponding PORT
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.append(proxy)
    return proxies

In [ ]:
def download_file(download_url,name, proxy):
    response = requests.get(download_url, proxies={"http": proxy, "https": proxy} )
    
    with open(f'{name}', 'wb') as f:
        f.write(response.content)
    f.close()

In [ ]:
import PyPDF2

In [ ]:
def pdf2str(pdf_link, fake_proxy):
  
    if fake_proxy:
        i = 0
        # I change the proxy until I can download the sentence
        while True:
            proxy = get_proxies()[i]
            try:
                download_file(pdf_link, 'sentenza.pdf', proxy)
                break
            except:
                i += 1
    else:
        #time.sleep(random.randint(1,5))
        !wget '{pdf_link}'  -O 'sentenza.pdf'
    pdfFileObj = open('sentenza.pdf', 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    text = ''
    for p in range(pdfReader.numPages):
        text +=  pdfReader.getPage(p).extractText()
    !rm 'sentenza.pdf'
    return text

In [ ]:
from urllib.parse import unquote

In [ ]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


In [ ]:
links = []

driver.get('http://www.italgiure.giustizia.it/sncass/')
time.sleep(random.randint(3,5))
# Click on 'sentenze' and 'penali' buttons
driver.find_element_by_id('4.[tipoprov]').click()
time.sleep(random.randint(4,7))
driver.find_element_by_id('2.[kind]').click()

for p in range(40):
    time.sleep(random.randint(1,5))
    cards = driver.find_elements_by_class_name('card')
    # unquote method is necessary because the link is encoded
    new_links = ['http://www.italgiure.giustizia.it' + unquote(x.find_elements_by_css_selector('a')[1].find_elements_by_css_selector('span')[0].find_element_by_css_selector('span').get_attribute('data-arg')) for x in cards]
    try:
        links.extend(new_links)
    except:
        pass
    time.sleep(random.randint(3,5))
    driver.find_element_by_class_name('pages').find_element_by_css_selector('span').click()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/selenium/webdriver/remote/webelement.py:502: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")
/usr/local/lib/python3.7/dist-packages/selenium/webdriver/remote/webelement.py:484: UserWarning: find_element_by_* commands are deprecated. Please use find_element

In [ ]:
len(links)

400

In [ ]:
fake_ip = False
i = 0
texts = []
new_links = []
for l in links:
    
    try:     
        texts.append(pdf2str(l, fake_proxy = fake_ip))
        new_links.append(l)
        print('ok: ', i)
        i += 1
    except:
        fake_ip = True

--2022-01-01 09:56:41--  http://www.italgiure.giustizia.it/xway/application/nif/clean/hc.dll?verbo=attach&db=snpen&id=./20211223/snpen@s20@a2021@n47125@tS.clean.pdf
Resolving www.italgiure.giustizia.it (www.italgiure.giustizia.it)... 89.119.251.144
Connecting to www.italgiure.giustizia.it (www.italgiure.giustizia.it)|89.119.251.144|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 457976 (447K) [application/pdf]
Saving to: ‘sentenza.pdf’

sentenza.pdf        100%[===================>] 447.24K   192KB/s    in 2.3s    

2022-01-01 09:56:44 (192 KB/s) - ‘sentenza.pdf’ saved [457976/457976]

ok:  0
--2022-01-01 09:56:45--  http://www.italgiure.giustizia.it/xway/application/nif/clean/hc.dll?verbo=attach&db=snpen&id=./20211223/snpen@s40@a2021@n47011@tS.clean.pdf
Resolving www.italgiure.giustizia.it (www.italgiure.giustizia.it)... 89.119.251.144
Connecting to www.italgiure.giustizia.it (www.italgiure.giustizia.it)|89.119.251.144|:80... connected.
HTTP request sent, awai

In [ ]:
df_sentenze = pd.DataFrame(
                            {'link': new_links,
                            
                            'text': texts
                            }
                           )

In [ ]:
df_sentenze.drop_duplicates(subset='text',inplace= True)

In [ ]:
df_sentenze.to_csv('/content/drive/MyDrive/TA_project/progetto_TA/NLP_Anonymization/data/sentenze_penale_pdf.csv',index = False)

In [ ]:
surplus_links = list(set(links)-set(new_links))

In [ ]:
len(surplus_links)

183

In [ ]:
fake_ip = False
i = 0
texts = []
new_links = []
for l in surplus_links:
    
    try:     
        texts.append(pdf2str(l, fake_proxy = fake_ip))
        new_links.append(l)
        print('ok: ', i)
        i += 1
    except:
        fake_ip = True

--2022-01-01 11:06:48--  http://www.italgiure.giustizia.it/xway/application/nif/clean/hc.dll?verbo=attach&db=snpen&id=./20211206/snpen@s40@a2021@n44985@tS.clean.pdf
Resolving www.italgiure.giustizia.it (www.italgiure.giustizia.it)... 89.119.251.144
Connecting to www.italgiure.giustizia.it (www.italgiure.giustizia.it)|89.119.251.144|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1141 (1.1K) [text/html]
Saving to: ‘sentenza.pdf’

sentenza.pdf        100%[===================>]   1.11K  --.-KB/s    in 0s      

2022-01-01 11:06:49 (131 MB/s) - ‘sentenza.pdf’ saved [1141/1141]

ok:  0
ok:  1
ok:  2
ok:  3
ok:  4
ok:  5
ok:  6
ok:  7
ok:  8


In [ ]:
df_sentenze_surplus = pd.DataFrame(
                            {'link': new_links,
                            
                            'text': texts
                            }
                           )

In [ ]:
df_sentenze_surplus.drop_duplicates(subset='text',inplace= True)

In [ ]:
df_sentenze_surplus.to_csv('/content/drive/MyDrive/TA_project/progetto_TA/NLP_Anonymization/data/sentenze_penale_pdf2.csv',index = False)

In [ ]:
df_sentenze_surplus

,link,text
0,http://www.italgiure.giustizia.it/xway/applica...,SENTENZA sul ricorso proposto da: BERATI MARSE...
1,http://www.italgiure.giustizia.it/xway/applica...,SENTENZA sul ricorso proposto da: Zaghali Abde...
2,http://www.italgiure.giustizia.it/xway/applica...,SENTENZA sul ricorso proposto da Mecja Albert ...
3,http://www.italgiure.giustizia.it/xway/applica...,SENTENZA sul ricorso proposto da: VILLANI GRET...
4,http://www.italgiure.giustizia.it/xway/applica...,SENTENZA sul ricorso proposto da Mastrogiacomo...
5,http://www.italgiure.giustizia.it/xway/applica...,SENTENZA sul ricorso proposto da Aliotta Andre...
6,http://www.italgiure.giustizia.it/xway/applica...,letta la requisitoria del Procuratore Generale...
7,http://www.italgiure.giustizia.it/xway/applica...,SENTENZA sul ricorso proposto dal Procuratore ...
8,http://www.italgiure.giustizia.it/xway/applica...,SENTENZA sul ricorso proposto da: KUMBULLA JET...
